# Análise exploratória de dados - GamersClub CS:GO

In [73]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import statsmodels.api as sm
import scipy.stats as stats


## Introdução

O jogo Counter-Strike Global Offensive, mais conhecido como CSGO, é um jogo de tiro em primeira pessoa (First Person Shooter), em que partidas são realizadas entre equipes de 5 jogadores cada. O jogo acaba quando uma das equipes ganha 16 rodadas. No caso de empate (por um placar de 15x15), são jogadas prorrogações com 6 rodadas até que uma das equipes ganhe 4 de 6 rodadas. O jogo compõe parte importante do cenário de esporte eletrônico, sendo acompanhado diariamente por milhares de espectadores em todos os países do mundo, apresentando cada vez mais espectadores no Brasil e atraindo uma quantidade considerável de investimentos.

A *GamersClub* é uma plataforma digital brasileira que tem como objetivo prover o serviço de criação de partidas para o CSGO, além de promover competições profissionais e aulas para aqueles que pagam pela assinatura do serviço.

O *dataset* utilizado nesse trabalho foi disponibilizado pela própria plataforma da *GamersClub* a partir do site do *Kaggle*. Esse conjunto de dados apresenta as informações de partidas jogadas por membros da plataforma, contendo informações de mais de 150 mil partidas. O mesmo pode ser obtido a partir do seguinte [endereço](https://www.kaggle.com/datasets/gamersclub/brazilian-csgo-plataform-dataset-by-gamers-club)

Os dados são apresentados de forma que cada linha represente uma partida de um determinado jogador. Dessa forma, todas as análises são feitas a nível individual, ou seja, não são analisados os dados de equipes, mas de jogadores. O conjunto de dados em si representa uma amostragem aleatória da plataforma. Nesse caso, a população analisada é o conjunto de **todos** os jogadores que participam da plataforma.

### Glossário

Utilitárias: elementos táticos do jogo, como:
- Granadas de fumaça, utilizadas para impedir a visão do time adversário
- Granada de fragmentação, utilizada para causar dano direto ao adversário através de explosão, sendo esse dano inversamente proporcional à distância da explosão para o jogador
- Granada incendiária, utilizada para impedir que jogadores adversários avancem sobre uma determinada região do mapa

ADR (*Average Damage per Round*): dano médio por rodada. Isto é, quanto um determinado jogador causou de dano na equipe inimiga a cada rodada, em média. Um jogador pode causar dano em um adversário a partir do uso de armas ou utilitárias (granadas).

HS (*Headshot*): um tiro que atinge a cabeça de um adversário. Estatística importante, pois é uma das maneiras mais eficientes de causar dano no adversário. À depender do armamento utilizado, causa uma eliminação instantânea.

## Parte 0: qualidade dos dados

In [74]:
lobby_players = pd.read_csv('data/tb_lobby_stats_player.csv')
# ignore players with vlLevel == 21
lobby_players = lobby_players[lobby_players['vlLevel'] != 21]

In [75]:
lobby_players.head(10)

,idLobbyGame,idPlayer,idRoom,qtKill,qtAssist,qtDeath,qtHs,qtBombeDefuse,qtBombePlant,qtTk,...,qtFlashAssist,qtHitHeadshot,qtHitChest,qtHitStomach,qtHitLeftAtm,qtHitRightArm,qtHitLeftLeg,qtHitRightLeg,flWinner,dtCreatedAt
0,1,1,1,5,1,16,2,0,0,0.0,...,0.0,3.0,13.0,4.0,2.0,2.0,1.0,0.0,0,2022-01-21 19:45:44
1,2,1,2,24,3,18,6,0,4,0.0,...,0.0,7.0,26.0,14.0,2.0,1.0,1.0,3.0,1,2022-02-04 02:09:47
2,3,2,3,6,4,23,2,0,1,0.0,...,0.0,3.0,15.0,8.0,1.0,2.0,0.0,2.0,0,2021-09-18 18:07:43
3,3,391,27508,10,5,20,4,1,0,0.0,...,0.0,6.0,27.0,10.0,1.0,7.0,6.0,6.0,1,2021-09-18 18:07:43
4,4,2,4,8,4,26,6,0,2,0.0,...,2.0,8.0,19.0,12.0,2.0,3.0,2.0,5.0,0,2021-09-27 00:17:45
5,5,2,5,10,1,11,5,0,3,0.0,...,1.0,5.0,6.0,8.0,0.0,0.0,3.0,0.0,1,2021-09-29 22:05:47
6,5,1068,69976,9,2,19,4,1,0,0.0,...,1.0,6.0,21.0,14.0,3.0,4.0,2.0,2.0,0,2021-09-29 22:05:47
7,6,2,6,16,1,23,9,0,1,0.0,...,1.0,9.0,21.0,19.0,3.0,4.0,2.0,3.0,0,2021-10-07 22:48:43
8,7,2,7,11,2,19,2,0,1,0.0,...,1.0,5.0,17.0,12.0,0.0,4.0,0.0,2.0,0,2021-10-08 23:29:57
9,8,2,8,8,6,13,2,0,0,0.0,...,0.0,3.0,9.0,17.0,2.0,5.0,7.0,8.0,0,2021-10-12 16:48:09


In [76]:
# count lines
print('Number of lines: ', len(lobby_players))
# check if all (idLobbyGame, idPlayer) are unique
print('Number of unique (idLobbyGame, idPlayer) pairs: ', len(lobby_players.groupby(['idLobbyGame', 'idPlayer']).size()))
# check if all (idRoom, idPlayer) are unique
print('Number of unique (idRoom, idPlayer) pairs: ', len(lobby_players.groupby(['idRoom', 'idPlayer']).size()))
# USAR APENAS O LOBBYGAME

Number of lines:  182798
Number of unique (idLobbyGame, idPlayer) pairs:  182798
Number of unique (idRoom, idPlayer) pairs:  182798


In [77]:
lobby_players['idLobbyGame'].value_counts().sort_values(ascending=False).head(10)

58891    5
17777    4
71305    4
60420    4
4508     4
17782    4
17622    4
93223    4
23787    3
9144     3
Name: idLobbyGame, dtype: int64

In [78]:
lobby_players['idRoom'].value_counts().sort_values(ascending=False).head(10)

1135      3
47171     3
17802     3
17803     3
108331    3
17813     3
17817     3
17818     3
17820     3
17794     3
Name: idRoom, dtype: int64

Conforme os dados acima, nota-se que não será possível realizar uma análise por equipes, uma vez que partidas com múltiplos jogadores simultâneas são escassas. Desta forma, a análise proposta tem como alvo principal o jogador individualmente, explorando informações como mapas, nível na GC, quantidade de eliminações, mortes e assistências, entre outras.

## Parte 1: players

In [79]:
players = pd.read_csv('data/tb_players.csv')

In [80]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idPlayer        2716 non-null   int64 
 1   flFacebook      2716 non-null   int64 
 2   flTwitter       2716 non-null   int64 
 3   flTwitch        2716 non-null   int64 
 4   descCountry     2716 non-null   object
 5   dtBirth         2008 non-null   object
 6   dtRegistration  2716 non-null   object
dtypes: int64(4), object(3)
memory usage: 148.7+ KB


In [81]:
players.head(10)

,idPlayer,flFacebook,flTwitter,flTwitch,descCountry,dtBirth,dtRegistration
0,1,0,0,0,cl,NaN,2021-03-19 21:31:39
1,2,0,0,0,br,NaN,2020-05-06 19:28:29
2,3,0,0,0,br,NaN,2020-01-28 13:00:38
3,4,0,0,0,br,NaN,2017-05-31 16:13:51
4,5,0,0,0,ar,NaN,2021-02-08 11:57:21
5,6,0,0,0,br,2002-03-23 00:00:00,2019-01-09 19:24:27
6,7,0,0,0,br,NaN,2020-04-24 22:53:50
7,8,0,0,0,br,NaN,2021-06-26 14:47:57
8,9,0,0,0,ar,2000-05-03 00:00:00,2016-05-22 15:12:48
9,10,0,0,0,br,2005-04-15 00:00:00,2018-02-28 11:29:11


In [82]:
print(players['flFacebook'].value_counts())
print(players['flTwitter'].value_counts())
print(players['flTwitch'].value_counts())

0    2653
1      63
Name: flFacebook, dtype: int64
0    2672
1      44
Name: flTwitter, dtype: int64
0    2663
1      53
Name: flTwitch, dtype: int64


Explicação das colunas:
* ``idPlayer``: id do player
* ``flFacebook``, ``flTwitter``, ``flTwitch``: se o jogador sincronizou (1) ou não (0) sua conta com cada uma das plataformas
* ``descCountry``: sigla de dois dígitos indicando o país do jogador
* ``dtRegistration``: data de ingresso do jogador na plataforma GamersClub

In [83]:
players['descCountry'].isna().sum()

0

Ou seja, todos os players tem um país.

In [84]:
countries_count = players['descCountry'].value_counts()
freq_countries = countries_count[countries_count > 100]
freq_countries

br    2071
ar     491
Name: descCountry, dtype: int64

In [85]:
# Plot to donut chart
fig = px.pie(countries_count, values=countries_count.values, names=countries_count.index, title='Países', width=800)
fig.update_traces(textposition='inside', textinfo='percent+label')
# # add hole
fig.update_traces(hole=.6, hoverinfo="label+percent+name")
fig.show()


In [86]:
# # Calculate the age of the players
players['age'] = (dt.datetime.now() - pd.to_datetime(players['dtBirth'])).astype('<m8[Y]')

In [87]:
# Histogram of player age
# Distribution curve over histogram
# Preparing data
x = np.array(players['age'])
# Remove NaN values
x = x[~np.isnan(x)]
# Remove x bigger than 47
x = x[x < 47]
print(x)
# Limit n bins to 20
fig = ff.create_distplot([x], ['Idade dos jogadores'], bin_size=1, show_rug=False, histnorm='probability', curve_type='normal')
# y = Densidade
# x = Idade
fig.update_layout(
    xaxis_title='Idade',
    yaxis_title='Densidade',
)
# fig width = 400px
fig.update_layout(width=800)
# Graph title = "Distribuição de idade dos jogadores da plataforma Gamers Club"
fig.update_layout(title='Distribuição de idade dos jogadores da plataforma Gamers Club')
# Set curve color to black
fig.update_traces(marker_color='#393E46')
# Set graph color to orange
fig.update_traces(selector=dict(type='histogram'), marker_color='orange')
fig.show()

[21. 23. 18. ... 23. 22. 23.]


**Nota: barras do histograma limitadas. Isso causou a remoção dos outliers: jogadores mais velhos que 45 anos. Haviam apenas 8.**

In [88]:
# boxplot
fig = px.box(players, y='age', title='Boxplot da idade dos jogadores da plataforma Gamers Club')
fig.update_layout(yaxis_title='Idade')
fig.show()

In [89]:
# mean
player_age_mean = players['age'].mean()
print('Média de idade dos jogadores: ', player_age_mean)

Média de idade dos jogadores:  24.786354581673308


In [90]:
# classify by skewness = mean - mode
player_age_skewness = player_age_mean - players['age'].mode()[0]
player_age_skewness

1.786354581673308

**Assimétrica à direita**

In [91]:
# coeficiente de assimetria de Pearson
player_age_skew_coeff = 3 * (players['age'].mean() - players['age'].median()) / players['age'].std()
player_age_skew_coeff

0.969757237080256

**Assimetria moderada**

In [92]:
player_age = np.array(players['age'])
# remove nan
player_age = player_age[~np.isnan(player_age)]
q3 = np.quantile(player_age, 0.75)
q1 = np.quantile(player_age, 0.25)
p90 = np.quantile(player_age, 0.90)
p10 = np.quantile(player_age, 0.10)
player_age_kurtosis = (q3 - q1) / (2 * (p90 - p10))
player_age_kurtosis

0.2692307692307692

**Platiocúrtica (ligeiramente)**

In [93]:
# How mah players in total
print(players['age'].count())
# How many players older than the mean age
print(players[players['age'] > player_age_mean]['age'].count())
# Percentage of players older than the mean age
print(players[players['age'] > player_age_mean]['age'].count() / players['age'].count())

2008
866
0.4312749003984064


Aproximadamente 43% dos players tem idade acima da média, o que explica a assimetria moderada à direita.

## Parte 2: resto das coisas

In [94]:
matches_level = lobby_players['vlLevel'].value_counts()
# bar plot
fig = px.bar(matches_level, x=matches_level.index, y=matches_level.values, title='Número de partidas por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Número de partidas', width=800)
fig.show()

In [95]:
# calculate correlation between level and number of matches
matches_level_corr = matches_level.corr(pd.Series(matches_level.index))
abs(matches_level_corr)

0.8853965023942966

Observa-se uma certa tendência nos dados, visto que a distribuição do número de partidas (cada linha da tabela é uma partida) é muito maior
para os níveis maiores (a correlação entre número de partidas jogadas e níveis é de, aproximadamente, 88%).

Contudo, trata-se de uma amostragem aleatória dos dados originais e, portanto, muito provavelmente refletem a realidade da população:
jogadores mais habilidosos (de maior nível) jogam mais partidas.
Tal fato faz sentido com a realidade: Counter-Strike exigie manutenção de habilidades (mira, por exemplo) -- jogadores de alto nível
que desejem manter-se assim tendem a jogar mais frequentemente.

In [96]:
# group by idPlayer, get the max vlLevel that the player reached
player_levels = lobby_players.groupby('idPlayer')['vlLevel'].max()
player_levels

idPlayer
1       10
2        3
3       15
4        0
5        2
        ..
2712    14
2713     6
2714    12
2715     7
2716     9
Name: vlLevel, Length: 2469, dtype: int64

### Distribuição de level

In [97]:
# Histogram of player level
# Distribution curve over histogram
# Preparing data
x = np.array(player_levels)
# GAMBIARRA: add element 21
x = np.append(x, 21)
print(x)
# Limit n bins to 20
fig = ff.create_distplot([x], ['Nível dos jogadores'], bin_size=1, show_rug=False, histnorm='probability density', curve_type='normal', )
# y = Densidade
# x = Idade
fig.update_layout(
    xaxis_title='Nível',
    yaxis_title='Probabilidade',
)
# fig width = 400px
fig.update_layout(width=800)
# Graph title = "Distribuição de idade dos jogadores da plataforma Gamers Club"
fig.update_layout(title='Distribuição de nível dos jogadores da plataforma Gamers Club')
# Set curve color to black
fig.update_traces(marker_color='#393E46')
# Set graph color to orange
fig.update_traces(selector=dict(type='histogram'), marker_color='orange')
# Remove legend
fig.update_layout(showlegend=False)
fig.show()

[10  3 15 ...  7  9 21]


In [98]:
# boxplot
fig = px.box(lobby_players, y='vlLevel', title='Boxplot do nível dos jogadores da plataforma Gamers Club')
fig.update_layout(yaxis_title='Nível')
fig.show()

In [99]:
# mean
player_level_mean = player_levels.mean()
print('Média de nível dos jogadores: ', player_level_mean)

Média de nível dos jogadores:  11.317942486836776


In [100]:
# get mode
player_level_mode = player_levels.mode()[0]
player_level_mode 

10

In [101]:
# classify by skewness
# skew calc: mean - mode
skew_calc = player_level_mean - player_level_mode
skew_calc

1.3179424868367757

Assimétrica à direita

In [102]:
player_level_skew_coeff = 3 * (lobby_players['vlLevel'].mean() - lobby_players['vlLevel'].median()) / lobby_players['vlLevel'].std()
abs(player_level_skew_coeff)

0.24850584681080948

Assimetria moderada

In [103]:
# kurtosis (q3 - q1) / 2(p90 - p10)
q3 = lobby_players['vlLevel'].quantile(0.75)
q1 = lobby_players['vlLevel'].quantile(0.25)
p90 = lobby_players['vlLevel'].quantile(0.90)
p10 = lobby_players['vlLevel'].quantile(0.10)
kurtosis_calc = (q3 - q1) / (2 * (p90 - p10))
kurtosis_calc

0.3076923076923077

Platicúrtica

A distribuição observada é a comum para jogos online: há poucos jogadores nos dois extremos.
Ou seja, pode-se dizer que a habilidade da maioria dos jogadores é mediana, havendo poucos jogadores que são muito ruins
e poucos que são muito bons. A moda, por exemplo, é exatamente o nível médio, 10 [nota de rodapé].

Nota de rodapé: Apesar dos níveis estarem distribuídos de 0 a 20, o nível 0 representa jogadores que ainda não cumpriram o período de calibração
da plataforma Gamers Club. 
Quando o jogador inscreve-se na plataforma, ela exige que o mesmo jogue 10 partidas antes de associar um nível
ao jogador.
Ao término dessas 10 partidas, um nível de 1 a 10 é associado ao jogador.

A assimetria moderada à direita pode ser facilmente explicada pelo fato da plataforma direcionar-se a jogadores mais avançados, que jogam profissionalmente ou como hobby "sério".
Dado isto, os jogadores da GC tendem a ser mais habilidosos do que aqueles que utilizam o sistema padrão de partidas do jogo, por exemplo.

### Mapas mais jogados

In [104]:
maps = lobby_players['descMapName'].value_counts()
# Plot em gráfico de barras
fig = px.bar(maps, x=maps.index, y=maps.values, title='Mapas')
fig.update_layout(xaxis_title='Mapa', yaxis_title='Quantidade de partidas', width=800)
fig.show()

In [105]:
# find out most recent date by column dtCreatedAt
print(lobby_players['dtCreatedAt'].max())
# find out oldest date by column dtCreatedAt
print(lobby_players['dtCreatedAt'].min())
# how many in 2021
print(lobby_players[lobby_players['dtCreatedAt'] >= '2021-01-01']['dtCreatedAt'].count())
# how many in 2022
print(lobby_players[lobby_players['dtCreatedAt'] >= '2022-01-01']['dtCreatedAt'].count())

2022-02-11 13:27:48
2021-09-14 13:06:52
182798
63087


Esta distribuição é explicada pelo fato da comunidade de Counter-Strike ser notadamene conservadora, isto é, resistente À mudanças.

Acient e Overpass são mapas exclusivos à última versão do jogo, Global Offensive.
Deste modo, houve uma resistência inicial muito grande da comunidade à estes mapas, que perdura até hoje, explicando o fato de não serem muito jogados.

Mirage, Inferno e Dust II, por outro lado, são mapas populares desde a primeira versão de Counter-Strike.

Vertigo, mesmo disponível desde a primeira versão de Counter-Strike, não se tornou popular até 2019, quando o mapa foi retrabalhado.

Train costumava ser um mapa popular, mas foi abandonado pelos campeonatos profissionais, não sendo mais atualizado e, portanto, tendo sua popularidade decaindo com o tempo.

Outro fator a ser considerado é o quão "pesado" os mapas são: mapas mais "pesados" exigem computadores mais podersos para executá-los.
Mirage é o mapa mais leve dentre os relatados.
Considerando que a grande maioria dos jogadores é brasileira, faz sentido que este seja o mapa mais jogado pois, economicamente falando,
pode-se assumir que boa parte destes jogadores não têm acesso a um computador de alta performance. [inserir dado de renda do Brasil]

Nuke é um mapa conhecidamente desequilibrado ([há vantagem para o lado Contraterrosita](https://draft5.gg/noticia/inferno-e-o-mapa-mais-balanceado-da-rotacao-nuke-e-onde-os-cts-tem-mais-vantagem)), sendo mais jogado em partidas profissionais ou amadoras sérias.
Este último fato levou o grupo a investigar a correlação entre mapas e os níveis dos jogadores.

### Mapas x níveis

In [106]:
# group by descMapName, levels and count
maps_levels = lobby_players.groupby(['descMapName', 'vlLevel']).size().reset_index(name='count')
maps_levels

,descMapName,vlLevel,count
0,de_ancient,0,8
1,de_ancient,1,2
2,de_ancient,2,9
3,de_ancient,3,39
4,de_ancient,4,44
...,...,...,...
163,de_vertigo,16,1446
164,de_vertigo,17,1378
165,de_vertigo,18,1599
166,de_vertigo,19,1861


In [107]:
# grouped bar chart
# y = count
# group by descMapName
# x = vlLevel
fig = px.bar(maps_levels, x='vlLevel', y='count', color='descMapName', title='Mapas por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Quantidade de partidas', width=800)
fig.show()

Surpreendentemente, como pode-se observar, há pouca diferença na distribuição dos mapas em realção aos níveis, isto é, a proporção se mantém.
Isto indica que os fatores extra-jogo provavelmente influenciam mais esta tendência, ou seja:
os jogadores, de modo geral, escolhem os mapas mais populares ou mais leves, 
não havendo uma relação clara entre o nível de habilidade do jogador e os mapas que ele preferem.
Isto também reforça a hipótese da relação entre o tradicionalismo da comunidade e suas preferências.

### Explorando outras hipóteses

Foi utilizada a média visto que, como observado, há muito mais partidas lvl 20 do que lvl menores. Logo a tendência é que os dados lvl 20 estourassem.

**Granadas por nível**

O bom uso de utilitários (granadas) é um dos principais diferenciais entre os times amadores e profissionais de CS:GO [citar HLTV].
Dado isto, o grupo decidiu analisar a correlação entre o nível dos jogadores e a quantidade de abates assisitidos de granadas de luz (*flash assists*), um dos dados disponíveis.
Abates assistidos por granadas de luz ocorrem quando um jogador mata outro logo após o adversário estar cego devido
a uma granada de luz.
Trata-se de um movimento comum em jogos profissionais: um jogador de suporte arremessa uma granada  de luz onde provavelmente há um inimigo,
enquanto outro jogador, inicialmente escondido, surge para abater o adversário assim que a granada explodir.

In [108]:
flash_assists_per_level = lobby_players.groupby('vlLevel')['qtFlashAssist'].mean()
flash_assists_per_level

vlLevel
0     0.339640
1     0.358974
2     0.346970
3     0.357112
4     0.380056
5     0.407234
6     0.387951
7     0.445423
8     0.432171
9     0.447828
10    0.466984
11    0.504138
12    0.511072
13    0.524980
14    0.531638
15    0.540748
16    0.580775
17    0.598279
18    0.651030
19    0.663683
20    0.896778
Name: qtFlashAssist, dtype: float64

In [109]:
# scatter plot
fig = px.scatter(flash_assists_per_level, x=flash_assists_per_level.index, y=flash_assists_per_level.values, title='Média de flash assists por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de flash assist por partida', width=800)
fig.show()

In [110]:
# correlation between flash assists and level
flash_assists_per_level.corr(pd.Series(flash_assists_per_level.index))

0.9181126961833269

Pode-se observar que a hipótese do grupo foi comprovada: há correlação fortíssima (aproximadamente 92%) entre o nível e a quantidade de abates
assistidos por granadas de luz.
Isto é explicado pela dificuldade de executar uma *flash assist*. Como esboçado, é necessário, no mínimo, dois jogadores
para executar uma jogada deste tipo. Ademais, isto exige uma comunicação e posicionamento precisos, caso contrário a jogada não será executada com
sucesso -- a granada será jogada antes do momento correto, a granada cegará o aliado ao invés do adversário, dentre outros erros comuns.
Portanto, faz sentido que a ocorrência destes eventos esteja relacionada a jogadores de mais alto nível.

faz sentido lvl 0 ter pouca flash assist pq provavelmente são partidas com random (flash assist é algo que requer coordenação do time)

**HS por nível**

...hs é uma mecanica muito importante, dano alto, dar mais hs faz parte do treinamento para tornar-se um jogador mais habilidoso...

In [111]:
hs_per_level = lobby_players.groupby('vlLevel')['qtHs'].mean()
hs_per_level

vlLevel
0     6.506149
1     4.354701
2     3.995455
3     4.457590
4     4.947075
5     5.347984
6     5.740923
7     5.989480
8     6.177857
9     6.598451
10    6.748830
11    6.998708
12    7.297038
13    7.485675
14    7.620147
15    7.838051
16    7.997027
17    8.138347
18    8.352915
19    8.834739
20    9.982023
Name: qtHs, dtype: float64

In [112]:
# scatter plot
fig = px.scatter(hs_per_level, x=hs_per_level.index, y=hs_per_level.values, title='Headshots por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de hs', width=800)
fig.show()

In [113]:
# correlation between hs and level
hs_per_level.corr(pd.Series(hs_per_level.index))

0.9277564709247353

hipótese comprovada: jogadores mais habilidosos tendem a dar mais hs

outlier lvl 0 pode ser explicado por jogadores habilidosos não classificados, isto é,
jogadores com experiência prévia ao ingresso da plataforma que, muito possivelmente,
foram classificados em um nível alto após a conclusão de suas 10 partidas classificatórias.

**Survived por nível**

[explicar porque guardar arma é importante](https://www.youtube.com/watch?v=UOFPycCdnC0&pp=ygUJamFtZSB0aW1l)

hipótese: jogadores mais habilidosos tendem a PRESERVAR (não guardar, o cara pode sobrevivier sem ganhar)
mais suas armas, ou seja, a tentar controlar melhor a economia do jogo

In [114]:
survived_per_level = lobby_players.groupby('vlLevel')['qtSurvived'].mean()
survived_per_level

vlLevel
0     6.506149
1     4.658120
2     5.250000
3     5.806003
4     6.035216
5     6.260994
6     6.448898
7     6.570047
8     6.619348
9     6.788365
10    6.890896
11    6.945517
12    7.004411
13    6.974295
14    6.988985
15    7.002254
16    7.086964
17    7.072220
18    7.156145
19    7.158467
20    7.326982
Name: qtSurvived, dtype: float64

In [115]:
# scatter plot
fig = px.scatter(survived_per_level, x=survived_per_level.index, y=survived_per_level.values, title='Sobrevivência por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de sobrevivência', width=800)
fig.show()

In [116]:
# correlation between survived and level
survived_per_level.corr(pd.Series(survived_per_level.index))

0.8234273621031272

a hipótese foi, de modo geral, comprovada: jogadores mais habilidosos tendem a preservar mais suas armas (correlação 82% aprox).
contudo, a curva mostra que os jogadores tendem a aprender a importância de preservar armamentos mais cedo em seu progresso no game:
níveis mais baixos já tem uma média considerável de sobreviência.

isso faz sentido, visto que sobreviver decorrer de uma simples ação no jogo, como afastar-se da explosão da bomba ou de regiões do
mapa onde conhecidamente há inimigos. ao contrário de dar HS, que é uma habilidade mecânica difícil de ser adquirida, e costuma melhorar
com o tempo; e de flash assists, que exigem plena coordenação de equipe.

outlier lvl 0 a explicação é a mesma de hs

### Coisas por round

**KPR por nível**

falar que KPR é uma medida ligeiramente superestimada -- CS é um jogo tático e há elementos mais importantes do que simplesmente eliminar um adversário

hipótese: KPR médio é equilibrado entre níveis, dado que as partidas costumam ser equilibradas entre jogadores (maioria tem nível parecido em uma partida)

In [117]:
lobby_players_with_kpr = lobby_players
lobby_players_with_kpr['kpr'] = lobby_players_with_kpr['qtKill'] / lobby_players_with_kpr['qtRoundsPlayed']
kpr_per_level = lobby_players_with_kpr.groupby('vlLevel')['kpr'].mean()
kpr_per_level

vlLevel
0     0.685599
1     0.500899
2     0.485156
3     0.530913
4     0.563113
5     0.600793
6     0.621815
7     0.634368
8     0.661749
9     0.683718
10    0.689434
11    0.698497
12    0.710437
13    0.712206
14    0.714907
15    0.731347
16    0.736990
17    0.743972
18    0.759091
19    0.772201
20    0.821574
Name: kpr, dtype: float64

In [118]:
fig = px.scatter(kpr_per_level, x=kpr_per_level.index, y=kpr_per_level.values, title='KPR por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de KPR', width=800)
fig.show()

In [119]:
# correlation between kpr and level
kpr_per_level.corr(pd.Series(kpr_per_level.index))

0.8805496570308828

hipótese NÃO se comprovou: há, sim, alta correlação entre KPR e nível

isto pode ser explicado por um dinâmica de partida: lvls mais altos tendem a ser mais consistentes, enquanto lvl mais baixos tendem a ser
do tipo "uns 2 amassam, o resto fica 0/todos".
infelizmente não temos dados consistentes de partidas inteiras para comprovar esta justificativa.

**Clutches por nível**

explicação

In [120]:
# clutches per level
clutches_per_level = lobby_players.groupby('vlLevel')['qtClutchWon'].mean()
clutches_per_level

vlLevel
0     0.436140
1     0.213675
2     0.287879
3     0.351022
4     0.378482
5     0.384655
6     0.389084
7     0.405968
8     0.445771
9     0.437439
10    0.443713
11    0.460942
12    0.462352
13    0.460567
14    0.456254
15    0.477679
16    0.472965
17    0.483140
18    0.500109
19    0.503259
20    0.541337
Name: qtClutchWon, dtype: float64

In [121]:
fig = px.scatter(clutches_per_level, x=clutches_per_level.index, y=clutches_per_level.values, title='Clutches por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de clutches', width=800)
fig.show()

hipótese não comprovada.

a média de clutches dos jogadores por partida é equilibrada entre níveis, com exceção dos níveis mais baixos,
onde, muito provavelmente, a maioria das rodadas encerra com 2 ou mais aliados vivos.

**Tk por level**

In [122]:
# qtTk per level
qtTk_per_level = lobby_players.groupby('vlLevel')['qtTk'].mean()
qtTk_per_level

vlLevel
0     0.021760
1     0.029915
2     0.031818
3     0.040452
4     0.033821
5     0.022639
6     0.029770
7     0.027933
8     0.026001
9     0.025759
10    0.027934
11    0.024914
12    0.021967
13    0.021193
14    0.022859
15    0.019370
16    0.019208
17    0.019193
18    0.019500
19    0.017330
20    0.020119
Name: qtTk, dtype: float64

In [123]:
# scatter plot
fig = px.scatter(qtTk_per_level, x=qtTk_per_level.index, y=qtTk_per_level.values, title='Team kills por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de team kills', width=800)
fig.show()

In [124]:
# correlation between qtTk and level
qtTk_per_level.corr(pd.Series(qtTk_per_level.index))

-0.7406083907041817

Há correlação inversa: níveis mais baixos tendem a mais TK, por um provável menor compromisso com o jogo

**Deaths por nível**

In [125]:
# dpr per level
lobby_players_with_dpr = lobby_players
lobby_players_with_dpr['dpr'] = lobby_players_with_dpr['qtDeath'] / lobby_players_with_dpr['qtRoundsPlayed']
dpr_per_level = lobby_players_with_dpr.groupby('vlLevel')['dpr'].mean()
dpr_per_level

vlLevel
0     0.705041
1     0.794436
2     0.764540
3     0.742318
4     0.738742
5     0.730601
6     0.726231
7     0.717667
8     0.716990
9     0.711783
10    0.709810
11    0.709504
12    0.709230
13    0.711190
14    0.712526
15    0.710907
16    0.709543
17    0.708862
18    0.706374
19    0.706967
20    0.694957
Name: dpr, dtype: float64

In [126]:
# scatter plot
fig = px.scatter(dpr_per_level, x=dpr_per_level.index, y=dpr_per_level.values, title='DPR por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de DPR', width=800)
fig.show()

In [127]:
# rounds played per level
rounds_played_per_level = lobby_players.groupby('vlLevel')['qtRoundsPlayed'].mean()
rounds_played_per_level

vlLevel
0     24.180700
1     23.136752
2     24.236364
3     24.853415
4     25.365251
5     25.625488
6     25.827126
7     25.605625
8     25.786475
9     25.946547
10    26.066009
11    26.376453
12    26.593282
13    26.542089
14    26.712524
15    26.603507
16    26.891219
17    26.844711
18    26.902722
19    26.897751
20    26.802087
Name: qtRoundsPlayed, dtype: float64

In [128]:
# scatter plot
fig = px.scatter(rounds_played_per_level, x=rounds_played_per_level.index, y=rounds_played_per_level.values, title='Rounds jogados por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de rounds jogados', width=800)
fig.show()

In [129]:
# line plots: kpr, dpr, rounds played
# multiple scales so we can see the difference between them~
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=kpr_per_level.index, y=kpr_per_level.values, name='KPR'), secondary_y=False)
fig.add_trace(go.Scatter(x=dpr_per_level.index, y=dpr_per_level.values, name='DPR'), secondary_y=False)
fig.add_trace(go.Scatter(x=rounds_played_per_level.index, y=rounds_played_per_level.values, name='Rounds jogados'), secondary_y=True)
fig.update_layout(xaxis_title='Nível', width=800)
fig.update_yaxes(title_text="KPR e DPR", secondary_y=False)
fig.update_yaxes(title_text="Rounds jogados", secondary_y=True)
fig.show()

In [130]:
# kills per level
kills_per_level = lobby_players.groupby('vlLevel')['qtKill'].mean()
kills_per_level

vlLevel
0     16.507096
1     11.794872
2     11.846970
3     13.221401
4     14.373259
5     15.418986
6     16.106348
7     16.265135
8     17.098773
9     17.772336
10    17.967982
11    18.425975
12    18.865103
13    18.928859
14    19.117043
15    19.513308
16    19.809191
17    19.985455
18    20.452748
19    20.800294
20    22.002459
Name: qtKill, dtype: float64

In [131]:
# scatter plot
fig = px.scatter(kills_per_level, x=kills_per_level.index, y=kills_per_level.values, title='Kills por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de kills', width=800)
fig.show()

In [132]:
# deaths per level
deaths_per_level = lobby_players.groupby('vlLevel')['qtDeath'].mean()
deaths_per_level

vlLevel
0     17.097446
1     18.145299
2     18.433333
3     18.349282
4     18.639972
5     18.708713
6     18.725931
7     18.381065
8     18.490801
9     18.481837
10    18.499927
11    18.723021
12    18.888027
13    18.864880
14    19.030213
15    18.888396
16    19.081144
17    19.034628
18    19.010655
19    19.010033
20    18.676193
Name: qtDeath, dtype: float64

In [133]:
# scatter plot
fig = px.scatter(deaths_per_level, x=deaths_per_level.index, y=deaths_per_level.values, title='Deaths por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de deaths', width=800)
fig.show()

In [134]:
# K/D per level
lobby_players_with_kd = lobby_players
lobby_players_with_kd['qtDeath'] = lobby_players_with_kd['qtDeath'].replace(0, 1)
lobby_players_with_kd['kd'] = lobby_players_with_kd['qtKill'] / lobby_players_with_kd['qtDeath']
kd_per_level = lobby_players_with_kd.groupby('vlLevel')['kd'].mean()
kd_per_level

vlLevel
0     1.145396
1     0.703414
2     0.700227
3     0.777381
4     0.820560
5     0.892360
6     0.927376
7     0.965281
8     0.991797
9     1.040456
10    1.049322
11    1.060605
12    1.076343
13    1.067847
14    1.069236
15    1.095262
16    1.106740
17    1.121654
18    1.149095
19    1.166431
20    1.269427
Name: kd, dtype: float64

In [135]:
# scatter plot
fig = px.scatter(kd_per_level, x=kd_per_level.index, y=kd_per_level.values, title='K/D por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de K/D', width=800)
fig.show()

In [136]:
# qtShots per level
qtShots_per_level = lobby_players.groupby('vlLevel')['qtShots'].mean()
qtShots_per_level

vlLevel
0     407.628193
1     291.794872
2     357.953030
3     388.733362
4     409.776114
5     412.583875
6     416.456781
7     424.580292
8     438.669948
9     454.227010
10    449.844737
11    447.094565
12    460.976459
13    450.641719
14    453.987087
15    464.000634
16    468.090464
17    465.593058
18    469.581187
19    469.950792
20    464.480466
Name: qtShots, dtype: float64

In [137]:
# scatter plot
fig = px.scatter(qtShots_per_level, x=qtShots_per_level.index, y=qtShots_per_level.values, title='Tiros por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de tiros', width=800)
fig.show()

In [138]:
# correlation between qtShots and level
qtShots_per_level.corr(pd.Series(qtShots_per_level.index))

0.8178913608135999

In [139]:
# qtTrade per level
qtTrade_per_level = lobby_players.groupby('vlLevel')['qtTrade'].mean()
qtTrade_per_level

vlLevel
0     2.635762
1     2.487179
2     2.881818
3     2.676816
4     2.857043
5     2.945095
6     2.917487
7     2.898582
8     2.863211
9     2.927308
10    2.978720
11    3.036810
12    3.063608
13    3.110475
14    3.156286
15    3.132211
16    3.215078
17    3.212856
18    3.179307
19    3.229878
20    3.243990
Name: qtTrade, dtype: float64

In [140]:
# scatter plot
fig = px.scatter(qtTrade_per_level, x=qtTrade_per_level.index, y=qtTrade_per_level.values, title='Trades por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de trades', width=800)
fig.show()

In [141]:
# diff per level
lobby_players_with_diff = lobby_players
lobby_players_with_diff['diff'] = lobby_players_with_diff['qtKill'] - lobby_players_with_diff['qtDeath']
diff_per_level = lobby_players_with_diff.groupby('vlLevel')['diff'].mean()	
diff_per_level

vlLevel
0    -0.590350
1    -6.350427
2    -6.586364
3    -5.127882
4    -4.266713
5    -3.290507
6    -2.620052
7    -2.116359
8    -1.392995
9    -0.709829
10   -0.532164
11   -0.297390
12   -0.023188
13    0.063818
14    0.086748
15    0.624560
16    0.727886
17    0.950496
18    1.441728
19    1.790133
20    3.326048
Name: diff, dtype: float64

In [142]:
# scatter plot
fig = px.scatter(diff_per_level, x=diff_per_level.index, y=diff_per_level.values, title='Diff por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de diff', width=800)
fig.show()

- HS por nível OK
- survived por nível OK
- KPR por nível (deve ser equilibrado, mas justificar)
- clutch por nível (hipótese: nível mais alto é mais difícil fazer clutch)

## Parte 3: índices e regressão

In [198]:
hltv_players = pd.read_csv('data/hltv/players.csv')
hltv_players.tail(10)

,name,dpr,kast,impact,adr,kpr,assists_per_round,rating
416,yekindar,0.70,68.8,1.29,82.6,0.74,0.13,1.14
417,yj,0.63,70.2,0.89,71.0,0.63,0.13,1.00
418,yuurih,0.64,73.2,1.14,83.9,0.75,0.15,1.18
419,zellsis,0.67,71.5,1.11,79.2,0.72,0.15,1.11
420,zeph,0.70,69.1,1.21,79.7,0.74,0.13,1.11
421,zevy,0.62,72.1,1.15,75.3,0.76,0.09,1.15
422,zorte,0.59,72.7,1.13,73.0,0.72,0.09,1.14
423,ztr,0.69,67.8,0.96,71.1,0.64,0.12,0.98
424,zyphon,0.66,71.1,1.05,75.9,0.69,0.12,1.07
425,zywoo,0.61,75.2,1.44,87.6,0.84,0.12,1.32


### Fórmula do Impact

In [199]:
y = hltv_players['impact']
x = hltv_players[['kpr', 'assists_per_round']]

# split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=666, test_size=0.2)

In [200]:
regr = LinearRegression()
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)

In [201]:
print(f'Coefficients: {regr.coef_}')
print(f'Intercept: {regr.intercept_}')
print()
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Coefficient of determination: {r2_score(y_test, y_pred)}')

Coefficients: [2.08326198 0.37503776]
Intercept: -0.4227985448322509

Mean squared error: 0.0018236240590910281
Coefficient of determination: 0.8028857398251624


In [202]:
inter_fmt = str(regr.intercept_).split('-')[1]
print('Fórmula do Impact:', regr.coef_[0], '* KPR +', regr.coef_[1], '* Assists por round -', inter_fmt)

Fórmula do Impact: 2.083261977948433 * KPR + 0.37503776257390314 * Assists por round - 0.4227985448322509


In [203]:
impact_kpr = regr.coef_[0]
impact_assists_per_round = regr.coef_[1]
impact_intercept = regr.intercept_
impact = lambda kpr, assists_per_round: impact_kpr * kpr + impact_assists_per_round * assists_per_round - impact_intercept

### Fórmula do Rating 2.0

In [204]:
y = hltv_players['rating']
x = hltv_players[['kast', 'kpr', 'dpr', 'impact', 'adr']]
x.head(1)

,kast,kpr,dpr,impact,adr
0,69.0,0.67,0.71,0.98,73.3


In [205]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=666, test_size=0.2)

In [206]:
regr = LinearRegression()
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)

In [207]:
print(f'Coefficients: {regr.coef_}')
print(f'Intercept: {regr.intercept_}')
print()
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Coefficient of determination: {r2_score(y_test, y_pred)}')

Coefficients: [ 0.00675127  0.36886234 -0.55817351  0.22974307  0.0035014 ]
Intercept: 0.20197217670601053

Mean squared error: 1.400327247010519e-05
Coefficient of determination: 0.995979822949647


In [208]:
print(f'Fórmula do Rating 2.0: {regr.coef_[0].round(5)} * KAST + {regr.coef_[1].round(5)} * KPR - {str(regr.coef_[2].round(5)).split("-")[1]} * DPR + {regr.coef_[3].round(5)} * Impact + {regr.coef_[4].round(5)} * ADR + {regr.intercept_.round(5)}')

Fórmula do Rating 2.0: 0.00675 * KAST + 0.36886 * KPR - 0.55817 * DPR + 0.22974 * Impact + 0.0035 * ADR + 0.20197


In [209]:
rating_kast = regr.coef_[0]
rating_kpr = regr.coef_[1]
rating_dpr = regr.coef_[2]
rating_impact = regr.coef_[3]
rating_adr = regr.coef_[4]
rating_intercept = regr.intercept_
rating = lambda kast, kpr, dpr, impact, adr: rating_kast * kast + rating_kpr * kpr - rating_dpr * dpr + rating_impact * impact + rating_adr * adr - rating_intercept

### Testando na GC

In [210]:
lobby_players_rating = lobby_players
lobby_players_rating['kpr'] = lobby_players_rating['qtKill'] / lobby_players_rating['qtRoundsPlayed']
lobby_players_rating['dpr'] = lobby_players_rating['qtDeath'] / lobby_players_rating['qtRoundsPlayed']
lobby_players_rating['kast'] = (lobby_players_rating['qtKill'] + lobby_players_rating['qtAssist'] + lobby_players_rating['qtTrade'] + lobby_players_rating['qtSurvived']) / lobby_players_rating['qtRoundsPlayed']
lobby_players['assists_per_round'] = lobby_players['qtAssist'] / lobby_players['qtRoundsPlayed']
lobby_players_rating['impact'] = impact(lobby_players['kpr'], lobby_players['assists_per_round'])
lobby_players_rating['adr'] = lobby_players_rating['vlDamage'] / lobby_players_rating['qtRoundsPlayed']

In [211]:
lobby_players_rating['rating'] = rating(lobby_players_rating['kast'], lobby_players_rating['kpr'], lobby_players_rating['dpr'], lobby_players_rating['impact'], lobby_players_rating['adr'])

In [212]:
lobby_players_rating['rating'].head(10)

0    0.866037
1    1.419522
2    0.786356
3    0.947563
4    0.818842
5    0.850266
6    1.100746
7    1.594663
8    1.065144
9    1.110614
Name: rating, dtype: float64

In [213]:
# rating per level
rating_per_level = lobby_players_rating.groupby('vlLevel')['rating'].mean()
rating_per_level

vlLevel
0     1.205912
1     1.019021
2     0.987600
3     1.030696
4     1.070573
5     1.112864
6     1.137199
7     1.149033
8     1.183902
9     1.209025
10    1.214301
11    1.226079
12    1.240812
13    1.243989
14    1.248396
15    1.267676
16    1.273888
17    1.282286
18    1.300007
19    1.317344
20    1.373066
Name: rating, dtype: float64

In [215]:
# scatter plot
fig = px.scatter(rating_per_level, x=rating_per_level.index, y=rating_per_level.values, title='Rating por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Média de rating 2.0', width=800)
fig.show()

In [218]:
# correlation between rating and level
# survived_per_level.corr(pd.Series(survived_per_level.index))
rating_per_level.corr(pd.Series(rating_per_level.index))

0.8923559280760794